In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# the service URL
livechart = "https://nds.iaea.org/relnsd/v0/data?"

# There have been cases in which the service returns an HTTP Error 403: Forbidden
# use this workaround
import urllib.request
def lc_pd_dataframe(url):
    req = urllib.request.Request(url)
    req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
    return pd.read_csv(urllib.request.urlopen(req))

In [2]:
# load data into a dataframe 
df = lc_pd_dataframe(livechart + "fields=decay_rads&nuclides=241am&rad_types=g").query("p_energy==0")

df = df[pd.to_numeric(df['intensity'],errors='coerce').notna()] # remove blanks (unknown intensities)
df.intensity = df['intensity'].astype(float) # convert to numeric. Note how one can specify the field by attribute or by string 


fig = px.scatter(df, x="energy", y="intensity", log_y=True) # plot in log scale
fig.show()


In [7]:
url=livechart+"fields=cumulative_fy&parents=235u"
df = lc_pd_dataframe(url) [['z_daughter','a_daughter',"element_daughter",'cumulative_fast_fy']]
df = df[pd.to_numeric(df.cumulative_fast_fy,errors='coerce').notna()]
df.cumulative_fast_fy = df.cumulative_fast_fy.astype(float)
  

fig = go.Figure(data=[go.Scatter3d(
    x=df["z_daughter"],
    y=df["a_daughter"]-df["z_daughter"],
    z=df["cumulative_fast_fy"],
    mode='markers',
    text = df['a_daughter'].apply(str) + df["element_daughter"],
    marker=dict(
        size=7
        ,color=df["cumulative_fast_fy"] # array/list of values setting the color
        ,colorscale='Viridis'           # colorscale
    )
     

)])

camera = dict(                          # initial spacial settings 
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=0.1, y=2, z=-1.5)        # view it from the bottom
)

fig.update_layout(scene_camera=camera, margin=dict(l=0, r=0, b=0, t=0), height=900)
fig.show()

# 2D plot
fig2 = go.Figure()
fig2.add_trace(go.Scatter( x=df[df.columns[1]], y=df['cumulative_fast_fy'], mode="markers", text= df['a_daughter'].apply(str) + df["element_daughter"]))
fig2.update_layout( height=600, xaxis_title="A", yaxis_title="Yield ") 
#fig2.show()
        

Cálculo de la Energía de ligadura $\Delta W$

In [4]:
def Energia_ligadura(A, Z, Masa_atomica): # en MeV
    energia = (Z*1.007825 + (A-Z)*1.008665-Masa_atomica)*931.5
    return f"La energía de ligadura es: {energia} MeV"


In [5]:
hidrogeno31 = Energia_ligadura(3, 1, 3.01604227)
hidrogeno31

'La energía de ligadura es: 8.488507995000013 MeV'

In [6]:
hidrogeno32 = Energia_ligadura(3,2, 3.016029)
hidrogeno32

'La energía de ligadura es: 7.718408999999601 MeV'

Energía de ligadura $\varepsilon$ por nucleón-Superificie de energía